<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/Mrc_BERT_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 3.0MB 15.7MB/s 
     |████████████████████████████████| 890kB 36.3MB/s 
     |████████████████████████████████| 1.1MB 27.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a02619cf3a529da7e433d475c54e4e57f0f0254d4fdb0a8663382419431f3266
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [20]:
from transformers import BertTokenizer, TFBertForQuestionAnswering
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForQuestionAnswering.from_pretrained("bert-base-cased")





Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertForQuestionAnswering: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[SEP] With a record 159 ##6 ##8 co ##rona ##virus cases reported in the past 24 hours , the total count in India crossed the 4 . 5 la ##kh


In [25]:
question, text = "what was the death toll" , "With a record 15968 coronavirus cases reported in the past 24 hours,the total count in India crossed the 4.5 lakh mark. The death toll has gone up to 14476"
input_dict = tokenizer(question, text, return_tensors='tf')
start_scores, end_scores = model(input_dict)
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_scores, 1)[0] : tf.math.argmax(end_scores, 1)[0]+1])
print(answer)

[SEP] With a record 159 ##6 ##8 co ##rona ##virus cases reported in the past 24 hours , the total count in India crossed the 4 . 5 la ##kh


In [12]:
model.summary()

Model: "tf_bert_for_question_answering_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
